### Pregunta 52: Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1, para cada grupo de usuarios que sabe una determinada cantidad de idiomas, ¿Cuántos de esos usuarios fueron bloqueados al menos una vez?

In [1]:
import pandas as pd
import numpy as np

In [2]:
columnas=["title","action"]
logs_df=pd.read_csv("logs.csv",usecols=columnas)
logs_df=logs_df.loc[logs_df['action']=="block",:]
logs_df.dropna(inplace = True)
logs_df['user_blocked'] = logs_df['title'].str.replace('Usuario:','') 

In [3]:
logs_df=logs_df.loc[:,["user_blocked"]]
logs_df['Fue_bloqueado']=1
blocked_users=logs_df.loc[:,['user_blocked','Fue_bloqueado']].drop_duplicates()

In [4]:
lenguaje_df=pd.read_csv("languages.csv")
lenguaje_df.dropna(inplace=True)
lenguaje_df['babel_level']=lenguaje_df['babel_level'].astype("string")
lenguaje_df=lenguaje_df.loc[lenguaje_df['babel_level']!="0",['babel_user','babel_level']]
lenguaje_df=lenguaje_df.groupby('babel_user').agg('count').reset_index()
lenguaje_df=lenguaje_df.rename(columns={'babel_level':'num_idiomas'})

In [5]:
usuarios_por_num_idiomas=lenguaje_df.groupby('num_idiomas').agg('sum').reset_index()

Se tiene el usuario de babel y el numero de idiomas distintos que conoce. Se necesita ahora vincularlo con aquellos usuarios que fueron bloqueados. 

Ahora que tengo todos los usernames de los usuarios bloqueados, hago join con el dataframe de (usuario, cantidad de idiomas conocidos)

In [28]:
columnas=['contributor_username','contributor_id']
id_df=pd.read_csv("logs.csv",usecols=columnas)
id_df.drop_duplicates(inplace=True)

Entiendo que el id del contributor en los logs es el mismo que el presente en el babel user. Puede ser que los datos que yo tengo de usuarios de babel no se correspondan todos con un valor de contributor id presente en los logs. Entonces deberia ver sobre cuales de ellos tengo informacion.

In [29]:
lenguaje_df['babel_user']=lenguaje_df['babel_user'].astype('object')

In [30]:
id_df=id_df.rename(columns={'contributor_id':'babel_user'})

In [31]:
babel_users_con_info_en_logs=lenguaje_df.merge(id_df, on='babel_user')

Conociendo usernames de usuarios de babel, puedo ahora vincularlos con los usuarios que fueron bloqueados

In [32]:
username_num_idiomas_bloqueados=babel_users_con_info_en_logs.loc[:,['babel_user','num_idiomas','contributor_username']]\
                                                            .rename(columns={'contributor_username':'user_blocked'})\
                                                            .merge(blocked_users,on='user_blocked')

In [33]:
username_num_idiomas_bloqueados=username_num_idiomas_bloqueados.loc[:,['num_idiomas','Fue_bloqueado']]\
                                                                .groupby('num_idiomas').agg('sum').reset_index()

In [ ]:
columnas=['contributor_ip','contributor_id']
ip_df=pd.read_csv("logs.csv",usecols=columnas)
ip_df.dropna(inplace=True)

In [ ]:
ip_df

El archivo de logs al parecer no tiene filas que posean tanto direccion ip como user id. Si las tuviera, se podria ejecutar el siguiente codigo (sabemos que entre los usuarios bloqueados hay direcciones ip)

In [23]:
#ips_num_idiomas_bloqueados=babel_users_con_info_en_logs.loc[:,['babel_user','num_idiomas','contributor_ip']]\
#                                                            .rename(columns={'contributor_ip':'user_blocked'})\
#                                                           .merge(blocked_users,on='user_blocked')

In [ ]:
#ips_num_idiomas_bloqueados=ips_num_idiomas_bloqueados.loc[:,['num_idiomas','Fue_bloqueado']]\
#                                                                .groupby('num_idiomas').agg('sum').reset_index()

In [ ]:
#usuarios_bloqueados_por_num_idiomas=pd.concat([username_num_idiomas_bloqueados,ips_num_idiomas_bloqueados])
#                                    .groupby('num_idiomas').agg('sum').reset_index()

Se tiene la cantidad de usuarios bloqueados a partir del username, asi que usamos eso para la resolucion

In [54]:
resolucion = usuarios_por_num_idiomas.merge(username_num_idiomas_bloqueados, on ='num_idiomas', how='left')
resolucion = resolucion.fillna(0)

In [57]:
resolucion=resolucion.rename(columns = {'Fue_bloqueado':'numero_usuarios_bloqueados','Babel_user':'numero_usuarios'})

In [58]:
resolucion

,num_idiomas,babel_user,numero_usuarios_bloqueados
0,1,1876733204,93.0
1,2,5326625659,182.0
2,3,4414525648,138.0
3,4,2600536694,82.0
4,5,1506392601,52.0
5,6,810684177,34.0
6,7,336506182,10.0
7,8,185925033,8.0
8,9,88954868,0.0
9,10,46497393,4.0
